### pico板
每10秒,儲存一次

### fast_api
多一個節點->https://xxxxxxxxxxx/temperature/xxxxxx

### redis
儲存一個key的名稱 board:temp
交程式和錄影檔

In [ ]:
# pico板 -每10秒,傳送訊息一次

from tools import connect, reconnect
from machine import ADC, Pin, Timer
import time
import urequests

connect()
adc = ADC(4)
conversion_factor = 3.3/65535

def aboard_temp(temp):
    reading_v = adc.read_u16() * conversion_factor
    celsius = round(27 - (reading_v-0.706) / 0.001721,2)
    url_str = f'https: // webServer-xxxx.onrender.com/temperature/{celsius}'
    try:
        response = urequests.get(url_str)
    except:
        print("ap出現問題")
        reconnect()
    else:
        if response.status_code == 200:
            print("傳送訊息成功")
        else:
            print("傳送失敗(make服務出問題)")
        response.close()


tim10 = Timer()
tim10.init(period=10*1000, callback=aboard_temp)


In [1]:
# fast_api - 多一個節點 -> https: // xxxxxxxxxxx/temperature/xxxxxx
# redis - 儲存一個key的名稱 board: temp

from typing import Union
from fastapi import FastAPI
import redis
import os
from dotenv import load_dotenv

celsius=27
load_dotenv()
redis_conn = redis.Redis.from_url(os.environ.get('REDIS_HOST_PASSWORD'))
redis_conn.set('board:temp', celsius)

app = FastAPI()

@app.get("/")
def read_root():
    return {"pico_w": "temperture"}


@app.get("/temperature/{celsius}")
def counter(c: int):
    celsius = redis_conn.get('board:temp')
    return {"temperature": celsius}


@app.get("/counter/{c}")
def counter(c: int):
    counter = redis_conn.incr('test:increment', c)
    return {"Counter": counter}


In [ ]:
# 接收pico_w傳來的資料(溫度、時間)
@app.get("/temperature/{celsius}/{time}")
def save_temperature(celsius: float, time: str):
    redis_conn.set('board:temperature', celsius)
    redis_conn.set('board:time', time)
    return 'Save successfully'

# 顯示溫度頁


@app.get("/temperature")
def read_temperature():
    celsius = redis_conn.get('board:temperature')
    time = redis_conn.get('board:time')
    return {"溫度": celsius, "時間": time}
